In [1]:
import os
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.llms import LlamaCpp


os.environ["OPENAI_API_KEY"]=""

In [2]:
# load multiple and process documents 
loader = DirectoryLoader('Pdf_Parsers_Txt_Files', glob="*.txt", loader_cls=TextLoader)

documents =loader.load()

In [3]:
#!pip install sentence_transformers

from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hfEmbedding = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

text = text_splitter.split_documents(documents)

print("length of text: ", len(text))

length of text:  1124


In [11]:
# create DB
directory = 'db'

doc_embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
                    documents = text,
                    embedding = hfEmbedding,
                    persist_directory= directory )

vectordb.persist()
vectordb = None

In [6]:
directory = 'db'
vectordb = Chroma(persist_directory = directory, embedding_function= hfEmbedding)

retriever = vectordb.as_retriever()

docs = retriever.get_relevant_documents("What is Forecast  Engine?")

print(len(docs))

#retriever = vectordb.as_retriever(search_kwargs = {"k":3})

#docs = retriever.get_relevant_documents("Basic Structure of the Local High")

#print(len(docs))

4


In [16]:
docs

[Document(page_content='● Smart  Flow  Manager\nAll the processed  data from forecast  engine  is used  here to evaluate  current  condition  of\nthe system and decisions such as creating backup and charge source during PV hours\nare taken  on the basis  of that.\n\nBasic Structure of the Local High\n\nWe are handling  slab and TOU  based  tariffs  in our first cut of the product.', metadata={'source': 'Pdf_Parsers_Txt_Files/Basic Structure of the Local High Voltage Product _parsed.txt'}),
 Document(page_content='5. periodic_comm  Handle Info (All the periodic calls,\nsuch as Load,PV,Grid,Inverter, Battery\nData)\n\nBasic Structure of the Local High\n\nSmartFlow\n\n● Forecast  Engine\nCombines  all the input  data such  as PV, load forecast  in a tabular  form with time slots.\nSaves  it in the database  and processes  this data.', metadata={'source': 'Pdf_Parsers_Txt_Files/Basic Structure of the Local High Voltage Product _parsed.txt'}),
 Document(page_content='energy management softw

In [ ]:
import torch
from transformers import pipeline
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from langchain.llms import HuggingFacePipeline

Tokenizer = LlamaTokenizer.from_pretrained("TheBloke/wizardLM-7B-HF")
model = LlamaForCausalLM.from_pretrained("TheBloke/wizardLM-7B-HF",
                                         load_in_8bit=True,
                                         device_map='auto',
                                         torch_dtype=torch.float16,
                                         low_cpu_mem_usage=True)

In [9]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "gpt2"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

In [10]:
pipe = pipeline(
                'text-generation',
                model=model,
                tokenizer=tokenizer,
                max_length=1024,
                temperature=0.3
)

local_llm = HuggingFacePipeline(pipeline=pipe)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [11]:
qa_chain = RetrievalQA.from_chain_type( 
                                        llm = local_llm, 
                                        chain_type="stuff", 
                                        retriever=retriever, 
                                        return_source_documents=True)

In [13]:
def process_llm_response(llm_response):
    print('Human Query: ', llm_response['query'])
    print("Assistant Response: ",llm_response['result'])


query = "What is Forecast  Engine?"

llm_response = qa_chain(query)
print(llm_response)

process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'query': 'What is Forecast  Engine?', 'result': ' Forecast  Engine is a program that can be used to generate  data from  forecast  engine  data. It is a tool that can be used to generate  data from  forecast  engine  data.\n\nIt is a tool that can be used to generate  data from  forecast  engine  data. It is a tool that can be used to generate  data from  forecast  engine  data. The tool can be used to generate  data from  forecast  engine  data. The tool can be used to generate  data from  forecast  engine  data. The tool can be used to generate  data from  forecast  engine  data. The tool can be used to generate  data from  forecast  engine  data. The tool can be used to generate  data from  forecast  engine  data. The tool can be used to generate  data from  forecast  engine  data. The tool can be used to generate  data from  forecast  engine  data. The tool can be used to generate  data from  forecast  engine  data. The tool can be used to generate  data from  forecast  engine  da